In [79]:
import numpy as np 
import pandas as pd
import math

# t=experiation time, p= ATM strike_price 4154.52
# p_c=call price, p_p=put price
# r=Risk-free interest rate to expiration
rf=0.016
time=30
T=time/365
strike_price=4154.52
def calculate_F(t,p,p_c,p_p,r):
    T=t/365
    F=p+math.exp(r*T)*(p_c-p_p)
    return F

In [22]:
data_type={'Strike':np.float64,"Midpoint":np.float64}
call=pd.read_csv("D://stock_data/call_option.csv",dtype=data_type)
put=pd.read_csv("D://stock_data/put_option.csv",dtype=data_type)
p_call=call["Midpoint"].median()
p_put=put["Midpoint"].median()
print(p_call,"/n",p_put)

forward_p=calculate_F(time,strike_price,p_call,p_put,rf)
forward_p

72.4 /n 63.125


4163.807205283908

In [85]:
K0=call[call["Strike"]<=forward_p]["Strike"].tail(1).values
kic=call[call["Strike"]>=forward_p][["Strike","Midpoint"]]
kip=put[put["Strike"]<forward_p][["Strike","Midpoint"]]
c=call[["Strike","Midpoint"]]
p=put[["Strike","Midpoint"]]
ki=pd.concat([kip,kic],axis=0)
deltaki=ki["Strike"].to_numpy()
lenth=len(deltaki)
delki=np.zeros(lenth)
for i in range(1,lenth-1):
    delki[i]=(deltaki[i+1]-deltaki[i-1])/2
delki[0]=deltaki[1]-deltaki[0]
delki[lenth-1]=deltaki[lenth-1]-deltaki[lenth-2]
delki
# ki
ki2=deltaki*deltaki
qki=(c["Midpoint"]+p["Midpoint"])/2
qki=qki.to_numpy()

In [93]:

def sum_ki(delki,ki2,r,T,qki):
    temp=0
    for i in range(len(delki)):
        temp+=delki[i]/ki2[i]*np.exp(r*T)*qki[i]
    
    return temp

In [96]:
sumi=sum_ki(delki,ki2,rf,T,qki)
theta=2/T*sumi-(1/T)*(forward_p/K0-1)**2
theta

array([0.00971582])